<a href="https://colab.research.google.com/github/ksetdekov/trip_choice_optimizer/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd

from  scipy.stats import gamma, norm

In [18]:
# np.random.seed(7336) # фиксируем seed

# risk tolerance
$$\rho = 3.37$$

# initialize

In [19]:
mu_hat_i = 0 # mean
t_i = 0
alpha_i = 0.5
beta_i = 0.5
# rho = 3.37
rho = 1

In [20]:
def update_mean(X, T_last, mu_last):
    mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
    return mu_new

def update_samples(T):
    return T + 1

def update_shape(a):
    return a + 0.5

def update_rate(X, mu_last, beta_last, T_last):
    beta_new = beta_last + (T_last / (T_last + 1)) * (np.square(X - mu_last)) / 2
    return beta_new

In [21]:
hands = pd.DataFrame({'name': ['трамвай + 944', 'с799 + пешком', 'пешком + 944', 'мотоцикл', 'с799 + 944', 
                               'рыжая + жёлтая + салатовая', 'трамвай + салатовая',
                               'каршеринг', 'серая + салатовая', 'рыжая + фиолетовая + салатовая', 'рыжая + салатовая',
                               'машина', 'самокат', '2гис', 'яндекс карты', 'гугл карты', 'citymapper', 'транспорт москвы']})

# 'московский велосипед'

In [22]:
hands['mu'] = 0
hands['Te'] = 0
hands['alpha'] = 0.5
hands['beta'] = 0.5

In [23]:
def to_minutes(timestr: str):
  '''
  convert timestr to float minutes
  '''
  return pd.to_timedelta(timestr).total_seconds()/60

In [24]:
def update_hands(hands, name, value):
    if isinstance(value, str):
        try:
            value = to_minutes(value)
        except ValueError:
            raise ValueError('input time string in hh:mm:ss format')
    elif isinstance(value, float) or isinstance(value, int):
        pass
    else:
        raise ValueError('input time string or int/float value')
    # print(value)
    _, mu, t, alpha, beta = hands[hands.name ==name].values[0]
    beta = update_rate(value, mu, beta, t)
    mu =  update_mean(value, t, mu)
    t = update_samples(t)
    alpha = update_shape(alpha)

    # print(mu, t, alpha, beta)

    hands.loc[hands.name ==name, 'mu'] = mu
    hands.loc[hands.name ==name, 'Te'] = t
    hands.loc[hands.name ==name, 'alpha'] = alpha
    hands.loc[hands.name ==name, 'beta'] = beta

    return hands


In [25]:
hands = update_hands(hands, 'мотоцикл', 20)
hands = update_hands(hands, 'рыжая + жёлтая + салатовая', 47.63)
hands = update_hands(hands, 'каршеринг', 31.75)
hands = update_hands(hands, 'рыжая + салатовая', 52.6)
hands = update_hands(hands, 'каршеринг', 23.15)
hands = update_hands(hands, 'рыжая + фиолетовая + салатовая', 52.82)
hands = update_hands(hands, 'серая + салатовая', 52.37)
hands = update_hands(hands, 'мотоцикл', 21.05)
hands = update_hands(hands, 'мотоцикл', 26.40)
hands = update_hands(hands, 'мотоцикл', "0:25:18")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:51")
hands = update_hands(hands, 'каршеринг', "0:28:04")
hands = update_hands(hands, 'каршеринг', "0:28:05")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:58")
hands = update_hands(hands, 'мотоцикл', "0:26:29")
hands = update_hands(hands, 'машина', "0:23:59")
hands = update_hands(hands, 'каршеринг', "0:36:41")
hands = update_hands(hands, 'машина', "0:24:49")
hands = update_hands(hands, 'машина', "0:21:56")
hands = update_hands(hands, 'машина', "0:22:03")
hands = update_hands(hands, 'машина', "0:23:21")
hands = update_hands(hands, 'мотоцикл', "0:27:45")
hands = update_hands(hands, 'машина', "0:22:26")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:24:07")
hands = update_hands(hands, 'машина', "0:23:38")
hands = update_hands(hands, 'машина', "0:24:13")
hands = update_hands(hands, 'машина', "0:25:32")
hands = update_hands(hands, 'машина', "0:22:11")
hands = update_hands(hands, 'самокат', "0:47:17")
hands = update_hands(hands, 'машина', "0:23:33")
hands = update_hands(hands, 'машина', "0:22:54")
hands = update_hands(hands, 'машина', "0:22:42")
hands = update_hands(hands, 'каршеринг', "0:33:02")
hands = update_hands(hands, 'машина', "0:23:31")
hands = update_hands(hands, 'машина', "0:24:39")
hands = update_hands(hands, 'машина', "0:24:16")
hands = update_hands(hands, 'машина', "0:28:12")
hands = update_hands(hands, 'трамвай + 944', "0:41:35")
hands = update_hands(hands, 'машина', "0:26:27")
hands = update_hands(hands, 'машина', "0:23:55")
hands = update_hands(hands, 'с799 + 944', "0:45:48")
hands = update_hands(hands, 'машина', "0:32:23")
hands = update_hands(hands, 'машина', "0:22:34")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:27:01")
hands = update_hands(hands, 'каршеринг', "0:31:14")
hands = update_hands(hands, 'трамвай + салатовая', "0:48:40")
hands = update_hands(hands, 'пешком + 944', "0:46:56")
hands = update_hands(hands, 'машина', "0:31:17")
hands = update_hands(hands, 'каршеринг', "0:34:08")
hands = update_hands(hands, 'машина', "0:31:18")
hands = update_hands(hands, 'машина', "0:27:10")
hands = update_hands(hands, 'каршеринг', "0:25:51")
hands = update_hands(hands, 'транспорт москвы', "0:49:36")
hands = update_hands(hands, 'яндекс карты', "0:44:21")
hands = update_hands(hands, 'с799 + пешком', "0:55:24")
hands = update_hands(hands, 'гугл карты', "0:40:50")
hands = update_hands(hands, '2гис', "0:59:04")
hands = update_hands(hands, 'каршеринг', "0:29:00")
hands = update_hands(hands, 'citymapper', "0:52:31")
hands = update_hands(hands, 'машина', "0:27:36")
hands = update_hands(hands, 'машина', "0:26:23")
hands = update_hands(hands, 'машина', "0:24:04")
hands = update_hands(hands, 'гугл карты', "0:44:06")
hands = update_hands(hands, 'машина', "0:33:59")
hands = update_hands(hands, 'каршеринг', "0:26:42")
hands = update_hands(hands, 'машина', "0:26:36")
hands = update_hands(hands, 'машина', "0:23:13")
hands = update_hands(hands, 'трамвай + 944', "0:48:14")
hands = update_hands(hands, 'машина', "0:25:05")
hands = update_hands(hands, 'машина', "0:30:01")
hands = update_hands(hands, 'каршеринг', "0:30:13")
hands = update_hands(hands, 'машина', "0:23:31")
hands = update_hands(hands, 'с799 + 944', "0:44:02")
hands = update_hands(hands, 'машина', "0:32:43")
hands = update_hands(hands, 'каршеринг', "0:25:07")
hands = update_hands(hands, 'гугл карты', "0:55:16")
hands = update_hands(hands, 'каршеринг', "0:35:15")
hands = update_hands(hands, 'с799 + 944', "0:56:38")
hands = update_hands(hands, 'машина', "0:23:55")
hands = update_hands(hands, 'машина', "0:22:29")
hands = update_hands(hands, 'яндекс карты', "0:47:40")
hands = update_hands(hands, 'каршеринг', "0:25:10")
hands = update_hands(hands, 'машина', "0:27:06")
hands = update_hands(hands, 'каршеринг', "0:30:21")
hands = update_hands(hands, 'машина', "0:23:50")
hands = update_hands(hands, 'машина', "0:21:28")
hands = update_hands(hands, 'машина', "0:26:38")
hands = update_hands(hands, 'машина', "0:21:33")
hands = update_hands(hands, 'машина', "0:25:22")
hands = update_hands(hands, 'каршеринг', "0:29:08")

hands

,name,mu,Te,alpha,beta
0,трамвай + 944,44.908333,2,1.5,11.555625
1,с799 + пешком,55.400000,1,1.0,0.500000
2,пешком + 944,46.933333,1,1.0,0.500000
3,мотоцикл,25.060000,10,5.5,30.344778
4,с799 + 944,48.822222,3,2.0,47.040370
5,рыжая + жёлтая + салатовая,47.630000,1,1.0,0.500000
6,трамвай + салатовая,48.666667,1,1.0,0.500000
7,каршеринг,29.583333,17,9.0,116.361111
8,серая + салатовая,52.370000,1,1.0,0.500000
9,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000


5: for each t = K + 1, K + 2, . . ., do

7: Sample τi,t from Gamma(αi,t−1, βi,t−1).

8: Sample θi,t from N (ˆµi,t−1, 1/Ti,t−1)

9: Play arm i(t) = arg maxi∈[K] ρθi,t − 1/τi,t and observe reward Xi(t),t

10: Update(ˆµi(t),t−1, Ti(t),t−1, αi(t),t−1, βi(t),t−1)

11: end for


In [26]:
tau = gamma.rvs( a = hands.alpha, scale=1/hands.beta)
theta = norm.rvs(hands.mu, 1/hands.Te)
hands['tau'] = tau
hands['theta'] = theta
hands['SD'] = np.sqrt(1/tau)
hands['var95'] = theta + norm.ppf(1-0.05/2) * hands.SD

if hands.mu.min() == 0:
    output1 = hands.reindex(np.argsort(hands.Te))
else:
    # hands['sorter'] = 1/tau + rho * theta
    output1 = hands.reindex(np.argsort(rho * theta + 1/tau))

output1

,name,mu,Te,alpha,beta,tau,theta,SD,var95
3,мотоцикл,25.060000,10,5.5,30.344778,0.344882,24.945745,1.702803,28.283179
11,машина,25.340217,46,23.5,217.779604,0.079911,25.339458,3.537496,32.272822
7,каршеринг,29.583333,17,9.0,116.361111,0.106920,29.535201,3.058234,35.529230
2,пешком + 944,46.933333,1,1.0,0.500000,2.044119,45.229164,0.699434,46.600030
5,рыжая + жёлтая + салатовая,47.630000,1,1.0,0.500000,2.059526,47.413187,0.696813,48.778916
12,самокат,47.283333,1,1.0,0.500000,1.082116,48.154202,0.961309,50.038333
6,трамвай + салатовая,48.666667,1,1.0,0.500000,0.394759,49.191823,1.591600,52.311301
14,яндекс карты,46.008333,2,1.5,3.250069,0.141101,45.268904,2.662161,50.486643
9,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000,3.218644,52.799670,0.557396,53.892145
16,citymapper,52.516667,1,1.0,0.500000,6.400214,53.056207,0.395278,53.830937


## для хилокомода

In [27]:
hands_drops = pd.DataFrame({'name': ['хилокомод', 'гилан', 'гилан ультра комфорт', 'стиллавит',
                                     'артелак баланс', 'артелак всплеск уно', 'артелак всплеск',
                                     'систейн ультра плюс', 'систейн ультра', 'визмед гель',
                                     'визмед лайт', 'хилабак', 'окутиарз'
                                     ]})
hands_drops['mu'] = 0
hands_drops['Te'] = 0
hands_drops['alpha'] = 0.5
hands_drops['beta'] = 0.5
# # с 5 по 20 декабря 2019
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(2/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(24/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # пропуск  с июля по октябрь 2020
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(30/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # март 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # апрель 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(20/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # # вышел на работу с сентября
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(21/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(32/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(16/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(10/12))
hands_drops = update_hands(hands_drops, 'гилан ультра комфорт', 30/(11/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(12/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10)) # начал Артелак всплеск
hands_drops = update_hands(hands_drops, 'артелак всплеск', 30/(12/10)) # начал визмед гель
hands_drops = update_hands(hands_drops, 'визмед гель', 30/(13/9)) # начал Артелак всплеск уно
hands_drops = update_hands(hands_drops, 'артелак всплеск уно', 30/(15/15)) # начал стиллавит
hands_drops = update_hands(hands_drops, 'стиллавит', 30/(8/10)) # начал артелак баланс - добавить к 
# стиллавиту столько дней, сколько использовал его потом
hands_drops = update_hands(hands_drops, 'артелак баланс', 30/(7/10)) # начал систейн ультра


hands_drops

,name,mu,Te,alpha,beta
0,хилокомод,24.525670,7,4.0,82.831148
1,гилан,23.378571,5,3.0,195.451020
2,гилан ультра комфорт,32.727273,1,1.0,0.500000
3,стиллавит,37.500000,1,1.0,0.500000
4,артелак баланс,42.857143,1,1.0,0.500000
5,артелак всплеск уно,30.000000,1,1.0,0.500000
6,артелак всплеск,25.000000,1,1.0,0.500000
7,систейн ультра плюс,0.000000,0,0.5,0.500000
8,систейн ультра,0.000000,0,0.5,0.500000
9,визмед гель,20.769231,1,1.0,0.500000


In [28]:
tau_drops = gamma.rvs( a = hands_drops.alpha, scale=1/hands_drops.beta)
theta_drops = norm.rvs(hands_drops.mu, 1/hands_drops.Te)
hands_drops['tau'] = tau_drops
hands_drops['theta'] = theta_drops
hands_drops['SD'] = np.sqrt(1/tau_drops)
hands_drops['var95'] = theta_drops + norm.ppf(1-0.05/2) * hands_drops.SD


if hands_drops.mu.min() == 0:
    
    output2 = hands_drops.reindex(np.argsort(hands_drops.Te))
else:
    # hands_drops['sorter'] = 1/tau_drops + rho * theta_drops
    output2 = hands_drops.reindex(np.argsort(rho * theta_drops + 1/tau_drops))

output2

# todo этот вариант кода переписать как опцию максимизации
# hands_drops['var95'] = theta_drops + norm.ppf(0.05/2) * hands_drops.SD


# if hands_drops.mu.min() == 0:
#     output2 = hands_drops.reindex(np.argsort(hands_drops.Te))
# else:
#     output2 = hands_drops.reindex(np.argsort(rho * theta_drops - 1/tau_drops)[::-1])


,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
7,систейн ультра плюс,0.000000,0,0.5,0.500000,0.638213,inf,1.251749,inf
8,систейн ультра,0.000000,0,0.5,0.500000,0.312172,-inf,1.789793,-inf
10,визмед лайт,0.000000,0,0.5,0.500000,0.001789,inf,23.641442,inf
11,хилабак,0.000000,0,0.5,0.500000,1.610485,-inf,0.787992,-inf
12,окутиарз,0.000000,0,0.5,0.500000,0.159876,inf,2.500969,inf
2,гилан ультра комфорт,32.727273,1,1.0,0.500000,1.923957,31.700877,0.720945,33.113904
3,стиллавит,37.500000,1,1.0,0.500000,0.728097,38.307334,1.171940,40.604293
4,артелак баланс,42.857143,1,1.0,0.500000,1.195957,45.237162,0.914413,47.029378
5,артелак всплеск уно,30.000000,1,1.0,0.500000,0.432248,30.101641,1.521014,33.082775


## для поездки с работы на ОТ, минимизуруем минуты

In [29]:
hands_back = pd.DataFrame({'name': ['2гис', 'яндекс карты', 'гугл карты', 'citymapper',
                                     'транспорт москвы'
                                     ]})
hands_back['mu'] = 0
hands_back['Te'] = 0
hands_back['alpha'] = 0.5
hands_back['beta'] = 0.5
hands_back = update_hands(hands_back, '2гис', '00:55:54')
hands_back = update_hands(hands_back, 'яндекс карты', '00:49:41')
hands_back = update_hands(hands_back, 'гугл карты', '01:05:54') # салатовая и 38 трамвай
hands_back = update_hands(hands_back, 'citymapper', '00:56:12')  # салатовая и 38 трамвай
hands_back = update_hands(hands_back, 'транспорт москвы', '00:51:29') # через третьяковскую до шаболовки
hands_back = update_hands(hands_back, '2гис', '01:06:55') # автобус до технопарка, от павелецкой - трамвай
hands_back = update_hands(hands_back, 'яндекс карты', '01:04:50') # 799 и 944 автобус


hands_back

,name,mu,Te,alpha,beta
0,2гис,61.408333,2,1.5,30.841736
1,яндекс карты,57.258333,2,1.5,57.880625
2,гугл карты,65.900000,1,1.0,0.500000
3,citymapper,56.200000,1,1.0,0.500000
4,транспорт москвы,51.483333,1,1.0,0.500000


In [30]:
tau_back = gamma.rvs( a = hands_back.alpha, scale=1/hands_back.beta)
theta_back = norm.rvs(hands_back.mu, 1/hands_back.Te)
hands_back['tau'] = tau_back
hands_back['theta'] = theta_back
hands_back['SD'] = np.sqrt(1/tau_back)
hands_back['var95'] = theta_back + norm.ppf(1-0.05/2) * hands_back.SD


if hands_back.mu.min() == 0:
    output3 = hands_back.reindex(np.argsort(hands_back.Te))
else:
    output3 = hands_back.reindex(np.argsort(rho * theta_back + 1/tau_back))

output3


,name,mu,Te,alpha,beta,tau,theta,SD,var95
4,транспорт москвы,51.483333,1,1.0,0.500000,4.143437,51.000066,0.491269,51.962936
3,citymapper,56.200000,1,1.0,0.500000,0.213475,56.028346,2.164346,60.270387
2,гугл карты,65.900000,1,1.0,0.500000,1.129406,66.245202,0.940968,68.089466
0,2гис,61.408333,2,1.5,30.841736,0.028086,61.287752,5.967007,72.982870
1,яндекс карты,57.258333,2,1.5,57.880625,0.023748,57.309689,6.489144,70.028176


## забота о себе

### уровень стресса за день после вечернего

In [31]:
journal_for_rest = pd.DataFrame({'name': ['ничего', 'gratitide journal бумажный', 
                                          'gratitide journal бумажный + woebot', 
                                          'woebot', 
                                          'gratitide journal бумажный + ReWi', 
                                          'ReWi'
                                     ]})
journal_for_rest['mu'] = -1
journal_for_rest['Te'] = 0
journal_for_rest['alpha'] = 0.5
journal_for_rest['beta'] = 0.5
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный', 43)
journal_for_rest = update_hands(journal_for_rest, 'ничего', 49)
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 34)  # на 26 января
journal_for_rest = update_hands(journal_for_rest, 'ничего', 31)  # на 27 января
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + ReWi', 37)  # на 28 января
journal_for_rest = update_hands(journal_for_rest, 'ReWi', 48)  # на 29 января
journal_for_rest = update_hands(journal_for_rest, 'woebot', 35)  # на 30 января
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 34)  # на 31 января
journal_for_rest = update_hands(journal_for_rest, 'woebot', 33)  # на 1 февраля
journal_for_rest = update_hands(journal_for_rest, 'woebot', 32)  # на 2 февраля
journal_for_rest = update_hands(journal_for_rest, 'woebot', 38)  # на 3 февраля 
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 28)  # на 4 февраля 
journal_for_rest = update_hands(journal_for_rest, 'ничего', 33) # ничего 5
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 33)  # gratitide journal бумажный + woebot 6
journal_for_rest = update_hands(journal_for_rest, 'ничего', 38) # ничего 7 и вечером gratitide journal бумажный + woebot
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 27)  # 8
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + ReWi', 30)  # 9
journal_for_rest = update_hands(journal_for_rest, 'ничего', 36) # ничего 10
journal_for_rest = update_hands(journal_for_rest, 'woebot', 35) #  11
journal_for_rest = update_hands(journal_for_rest, 'ничего', 39) #  12
journal_for_rest = update_hands(journal_for_rest, 'ничего', 32) #  13
journal_for_rest = update_hands(journal_for_rest, 'ничего', 31) #  14 
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 31)  # 15 
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 33)  # 16
journal_for_rest = update_hands(journal_for_rest, 'ничего', 32) #  17
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 32)  # 18
journal_for_rest = update_hands(journal_for_rest, 'woebot', 33) #  19
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 32)  # 20
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 39)  # 21
journal_for_rest = update_hands(journal_for_rest, 'woebot', 38) #  22
journal_for_rest = update_hands(journal_for_rest, 'ничего', 47) #  23
journal_for_rest = update_hands(journal_for_rest, 'woebot', 34) #  24
journal_for_rest = update_hands(journal_for_rest, 'woebot', 27) #  25
journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + ReWi', 23) #  26
journal_for_rest = update_hands(journal_for_rest, 'ничего', 27)  # 27
journal_for_rest = update_hands(journal_for_rest, 'woebot', 40)  # 28
journal_for_rest = update_hands(journal_for_rest, 'woebot', 29)  # 1 march
# journal_for_rest = update_hands(journal_for_rest, 'woebot', 39)  # 2 march
# journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный + woebot', 32)  # 3
# journal_for_rest = update_hands(journal_for_rest, 'gratitide journal бумажный', 43) # 4


journal_for_rest

,name,mu,Te,alpha,beta
0,ничего,35.909091,11,6.0,237.954545
1,gratitide journal бумажный,43.000000,1,1.0,0.500000
2,gratitide journal бумажный + woebot,32.300000,10,5.5,50.550000
3,woebot,34.000000,11,6.0,75.500000
4,gratitide journal бумажный + ReWi,30.000000,3,2.0,49.500000
5,ReWi,48.000000,1,1.0,0.500000


In [32]:
tau_journal = gamma.rvs( a = journal_for_rest.alpha, scale=1/journal_for_rest.beta)
theta_journal = norm.rvs(journal_for_rest.mu, 1/journal_for_rest.Te)
journal_for_rest['tau'] = tau_journal
journal_for_rest['theta'] = theta_journal
journal_for_rest['SD'] = np.sqrt(1/tau_journal)
journal_for_rest['var95'] = theta_journal + norm.ppf(1-0.05/2) * journal_for_rest.SD


if journal_for_rest.mu.min() == -1:
    output4 = journal_for_rest.reindex(np.argsort(journal_for_rest.Te))
else:
    output4 = journal_for_rest.reindex(np.argsort(rho * theta_journal + 1/tau_journal))

output4


,name,mu,Te,alpha,beta,tau,theta,SD,var95
1,gratitide journal бумажный,43.000000,1,1.0,0.500000,1.587394,41.069410,0.793702,42.625038
2,gratitide journal бумажный + woebot,32.300000,10,5.5,50.550000,0.087641,32.171449,3.377888,38.791988
5,ReWi,48.000000,1,1.0,0.500000,2.396239,49.011152,0.646004,50.277296
3,woebot,34.000000,11,6.0,75.500000,0.053662,34.006193,4.316857,42.467077
0,ничего,35.909091,11,6.0,237.954545,0.029200,35.946343,5.852068,47.416185
4,gratitide journal бумажный + ReWi,30.000000,3,2.0,49.500000,0.003174,29.492477,17.750692,64.283193
